In [15]:
#Importing dependencies/libraries

from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
import time
import numpy as np

In [2]:
#ON
location_on = ["greater-sudbury", "hamilton", "kingston", "london", "niagara-falls",
            "ottawa", "toronto", "windsor", "kitchener", "peterborough", 
            "sault-ste-marie", "thunder-bay"]

#AB
location_ab = ["edmonton", "calgary", "lethbridge", "red-deer"]

#NS
location_ns = ["halifax"]

#QC
location_qc = ["montreal", "sherbrooke", "trois-rivieres", "quebec"]

#BC
location_bc = ["vancouver", "victoria"]

#MB
location_mb = ["winnipeg"]

pages = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230]

In [3]:
#empty lists to hold the values
names = []
price_range = []
ratings = []
total_reviews = []
location = []
province = []

#web scraping
browser = Browser('chrome')
#This for loop changes the page on the url
for loc in location_bc:  
    #This for loop changes the location of the url
    for page in pages:  
        
        url = f'https://www.yelp.com/search?find_desc=Restaurants&find_loc={loc}%2C+bc&start={page}'
        browser.visit(url)
        html = browser.html
        soup_obj = soup(html, 'html.parser')      
        
        #the idea is to extract the div that contains the elements I want and saving it into res_info
        res_info = []    
        res_info = soup_obj.find_all("div", class_ = "container__09f24__mpR8_ hoverable__09f24__wQ_on border-color--default__09f24__NPAKY")
        time.sleep(5)
        
        #extracting the elements from res_info
        for info in res_info:
            #extracting restaurant name
            try:
                name = info.find("a", class_ = "css-19v1rkv").text
                names.append(name)
            except:
                names.append("No data or change the code")
              
            #extracting price range
            if info.find("span", {"class":"priceRange__09f24__mmOuH css-chan6m"}):
                prc_range = info.find("span", {"class":"priceRange__09f24__mmOuH css-chan6m"}).text
                price_range.append(prc_range)
            elif info.find("span", {"class":"priceRange__09f24__mmOuH css-1s7bx9e"}):
                prc_range = info.find("span", {"class":"priceRange__09f24__mmOuH css-1s7bx9e"}).text
                price_range.append(prc_range)
            else:
                price_range.append("No data or change the code")
            
            #extracting ratings
            try:
                rati = info.find("span", {"class":"css-gutk1c"}).text
                ratings.append(rati)
            except:
                ratings.append("No data or change the code")
            
            #extracting total number of reviews
            try:
                revs = info.find("span", {"class":"css-chan6m"}).text
                total_reviews.append(revs)
            except:
                total_reviews.append("No data or change the code")
            
            location.append(loc)
            province.append("BC")
               

In [4]:
len(names)

624

In [9]:
len(price_range)

624

In [60]:
yelp_bc_df = pd.DataFrame({
    "restaurant_name" : names,
    "price_range" : price_range,
    "rating" : ratings,
    "total_reviews" : total_reviews,
    "location" : location,
    "province" : province
})

yelp_bc_df

,restaurant_name,price_range,rating,total_reviews,location,province
0,Pizza Hut,No data or change the code,No data or change the code,Riley Park,vancouver,BC
1,Chessey Pizza Corner,No data or change the code,No data or change the code,,vancouver,BC
2,The Flying Pig,$$,4.0,(1.3k reviews),vancouver,BC
3,Kingyo,$$,4.4,(1.1k reviews),vancouver,BC
4,Miku,$$$,4.4,(2.2k reviews),vancouver,BC
...,...,...,...,...,...,...
619,BobaDog Bubble Tea & Hotdog,No data or change the code,3.5,(4 reviews),victoria,BC
620,Vis-à-Vis,$$,4.1,(39 reviews),victoria,BC
621,Shine Cafe,$$,4.1,(68 reviews),victoria,BC
622,Loft and Ladder,$,4.7,(6 reviews),victoria,BC


In [61]:
yelp_bc_df.drop_duplicates()
yelp_bc_df.replace('No data or change the code', np.nan, inplace = True)
yelp_bc_df.dropna(inplace = True)
yelp_bc_df = yelp_bc_df.reset_index(drop = True)
yelp_bc_df['total_reviews'] = yelp_bc_df['total_reviews'].str.replace(r'[\(\) reviews]', '', regex=True)

yelp_bc_df

,restaurant_name,price_range,rating,total_reviews,location,province
0,The Flying Pig,$$,4.0,1.3k,vancouver,BC
1,Kingyo,$$,4.4,1.1k,vancouver,BC
2,Miku,$$$,4.4,2.2k,vancouver,BC
3,Kosoo,$$,4.3,88,vancouver,BC
4,Dinesty Dumpling House,$$,3.9,928,vancouver,BC
...,...,...,...,...,...,...
317,Pho Vy Vietnamese Restaurant,$$,3.6,85,victoria,BC
318,Avalon Restaurant,$$,4.1,33,victoria,BC
319,Vis-à-Vis,$$,4.1,39,victoria,BC
320,Shine Cafe,$$,4.1,68,victoria,BC


In [68]:
yelp_bc_df.to_csv("yelp_bc.csv", index = False)

In [ ]:
browser.quit()